In [10]:
# Context Manager

class File:
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
        
    def __enter__(self):
        print('opening file')
        self.file = open(self.name, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('closing file')
        self.file.close()
        return False

In [11]:
with File('data/test.txt', 'w') as f:
    f.write('This is a test')

opening file
closing file


In [12]:
with File('data/test.txt', 'r') as f:
    print(f.readlines())

opening file
['This is a test']
closing file


In [13]:
# Context Manager with Generator

class GenContextManager:
    def __init__(self, gen, *args, **kwargs):
        self._gen = gen(*args, **kwargs)
        
    def __enter__(self):
        return next(self._gen)
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False
    
def open_file(fname, mode):
    f = open(fname, mode)
    try:
        print('Opening file')
        yield f
    finally:
        print('Closing file')
        f.close()

In [17]:
with GenContextManager(open_file, 'data/test.txt', 'r') as f:
    print(f.readlines())

Opening file
['This is a test']
Closing file


In [24]:
# Context Manager with Decorator and Generator

class GenContextManager:
    def __init__(self, gen):
        self._gen = gen
        
    def __enter__(self):
        return next(self._gen)
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False

    
def context_manager_dec(fgen):
    def helper(*args, **kwargs):
        gen = fgen(*args, **kwargs)
        ctx = GenContextManager(gen)
        return ctx
    return helper


@context_manager_dec
def open_file(fname, mode = 'r'):
    f = open(fname, mode)
    try:
        print('Opening file')
        yield f
    finally:
        print('Closing file')
        f.close()

In [26]:
with open_file('data/test.txt') as f:
    print(f.readlines())

Opening file
['This is a test']
Closing file
